# Haematobium Analysis

In [161]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import warnings
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from xgboost import XGBRegressor, plot_importance

In [239]:
df = pd.read_csv("sh.csv")

Geophysical Group

In [163]:
geophysical_vars = ['aspect', 'bedrock_depth_mean', 'bulk_density_20cm_mean', 'cec_20cm_mean', 'clay_20cm_mean', 
                    'damDist', 'elevation', 'hnd', 'nitrogen_20cm_mean', 'organic_carbon_20cm_mean', 'ph_20cm_mean',
                    'sand_20cm_mean', 'silt_20cm_mean', 'slope', 'soilWater', 'stone_20cm_mean', 'total_carbon_20cm_mean',
                    'upa', 'waterDist'] 

target_var = 'sh_prev'

In [164]:
X_geophys = df[geophysical_vars] #input features
y = df[target_var] #target var - shprev

In [165]:
X_geophystrain, X_geophystest, y_train, y_test = train_test_split(X_geophys, y, test_size=0.2, random_state=42)

In [166]:
#Linear Regressor for Geophys, evaluation with RMSE direct.
geophys_pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
geophys_pipeline_LR.fit(X_geophystrain, y_train)
y_geophys_pred = geophys_pipeline_LR.predict(X_geophystest)
rmse = np.sqrt(mean_squared_error(y_test, y_geophys_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 10.557702380233529


In [167]:
#Linear Regressor for Geophys, evaluation with Cross-val
rmse_scorer = make_scorer(mean_squared_error, squared=False)
cv_scores = cross_val_score(geophys_pipeline_LR, X_geophys, y, cv=5, scoring=rmse_scorer)
mean_rmse = np.mean(cv_scores)
print(f"Mean RMSE: {mean_rmse}")
warnings.filterwarnings("ignore", category=DeprecationWarning)

Mean RMSE: 12.62365693563078


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [168]:
#XGBoost Regressor for Geophys, evaluation with RMSE direct.
geophys_pipeline_XGB = Pipeline([
    ('scaler', StandardScaler()),  
    ('regressor', XGBRegressor()) 
])
geophys_pipeline_XGB.fit(X_geophystrain, y_train)
y_geophys_pred = geophys_pipeline_XGB.predict(X_geophystest)
rmse = np.sqrt(mean_squared_error(y_test, y_geophys_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 12.973830926471912


In [169]:
#XGBoost Regressor for Geophys, evaluation with Cross-val
rmse_scorer = make_scorer(mean_squared_error, squared=False)
cv_scores = cross_val_score(geophys_pipeline_XGB, X_geophys, y, cv=10, scoring=rmse_scorer)
mean_rmse = np.mean(cv_scores)
print(f"Mean RMSE: {mean_rmse}")
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 12.711086899234198


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [170]:
#Feature importance in Regressor
xgb_model = geophys_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_geophystrain.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df

,Feature,Importance
3,cec_20cm_mean,0.113007
6,elevation,0.111843
8,nitrogen_20cm_mean,0.098723
14,soilWater,0.068238
13,slope,0.059811
12,silt_20cm_mean,0.053686
1,bedrock_depth_mean,0.052844
4,clay_20cm_mean,0.052633
2,bulk_density_20cm_mean,0.051545
17,upa,0.047956


Climate Group

In [171]:
aet_mean_laggers = ["aet_mean_lag1mo", "aet_mean_lag2mo", "aet_mean_lag3mo", "aet_mean_lag4mo", "aet_mean_lag5mo", "aet_mean_lag6mo",
                    "aet_mean_lag7mo", "aet_mean_lag8mo", "aet_mean_lag9mo", "aet_mean_lag10mo", "aet_mean_lag11mo", "aet_mean_lag12mo"]

def_mean_laggers = ["def_mean_lag1mo", "def_mean_lag2mo", "def_mean_lag3mo", "def_mean_lag4mo", "def_mean_lag5mo", "def_mean_lag6mo",
                    "def_mean_lag7mo", "def_mean_lag8mo", "def_mean_lag9mo", "def_mean_lag10mo", "def_mean_lag11mo", "def_mean_lag12mo"]

pdsi_mean_laggers = ["pdsi_mean_lag1mo", "pdsi_mean_lag2mo", "pdsi_mean_lag3mo", "pdsi_mean_lag4mo", "pdsi_mean_lag5mo", "pdsi_mean_lag6mo",
                     "pdsi_mean_lag7mo", "pdsi_mean_lag8mo", "pdsi_mean_lag9mo", "pdsi_mean_lag10mo", "pdsi_mean_lag11mo", "pdsi_mean_lag12mo"]

pet_mean_laggers = ["pet_mean_lag1mo", "pet_mean_lag2mo", "pet_mean_lag3mo", "pet_mean_lag4mo", "pet_mean_lag5mo", "pet_mean_lag6mo", 
                    "pet_mean_lag7mo", "pet_mean_lag8mo", "pet_mean_lag9mo", "pet_mean_lag10mo", "pet_mean_lag11mo", "pet_mean_lag12mo"]

pr_mean_laggers = ["pr_mean_lag1mo", "pr_mean_lag2mo", "pr_mean_lag3mo", "pr_mean_lag4mo", "pr_mean_lag5mo", "pr_mean_lag6mo", 
                   "pr_mean_lag7mo", "pr_mean_lag8mo", "pr_mean_lag9mo", "pr_mean_lag10mo", "pr_mean_lag11mo", "pr_mean_lag12mo"]

ro_mean_laggers = ["ro_mean_lag1mo", "ro_mean_lag2mo", "ro_mean_lag3mo", "ro_mean_lag4mo", "ro_mean_lag5mo", "ro_mean_lag6mo", 
                   "ro_mean_lag7mo", "ro_mean_lag8mo", "ro_mean_lag9mo", "ro_mean_lag10mo", "ro_mean_lag11mo", "ro_mean_lag12mo"]

soil_mean_laggers = ["soil_mean_lag1mo", "soil_mean_lag2mo", "soil_mean_lag3mo", "soil_mean_lag4mo", "soil_mean_lag5mo", "soil_mean_lag6mo",
                     "soil_mean_lag7mo", "soil_mean_lag8mo", "soil_mean_lag9mo", "soil_mean_lag10mo", "soil_mean_lag11mo", "soil_mean_lag12mo"]

srad_mean_laggers = ["srad_mean_lag1mo", "srad_mean_lag2mo", "srad_mean_lag3mo", "srad_mean_lag4mo", "srad_mean_lag5mo", "srad_mean_lag6mo", 
                     "srad_mean_lag7mo", "srad_mean_lag8mo", "srad_mean_lag9mo", "srad_mean_lag10mo", "srad_mean_lag11mo", "srad_mean_lag12mo"]

tmmn_mean_laggers = ["tmmn_mean_lag1mo", "tmmn_mean_lag2mo", "tmmn_mean_lag3mo", "tmmn_mean_lag4mo", "tmmn_mean_lag5mo", "tmmn_mean_lag6mo", 
                     "tmmn_mean_lag7mo", "tmmn_mean_lag8mo", "tmmn_mean_lag9mo", "tmmn_mean_lag10mo", "tmmn_mean_lag11mo", "tmmn_mean_lag12mo"]

tmmx_mean_laggers = ["tmmx_mean_lag1mo", "tmmx_mean_lag2mo", "tmmx_mean_lag3mo", "tmmx_mean_lag4mo", "tmmx_mean_lag5mo", "tmmx_mean_lag6mo",
                     "tmmx_mean_lag7mo", "tmmx_mean_lag8mo", "tmmx_mean_lag9mo", "tmmx_mean_lag10mo", "tmmx_mean_lag11mo", "tmmx_mean_lag12mo"]

vap_mean_laggers = ["vap_mean_lag1mo", "vap_mean_lag2mo", "vap_mean_lag3mo", "vap_mean_lag4mo", "vap_mean_lag5mo", "vap_mean_lag6mo", 
                    "vap_mean_lag7mo", "vap_mean_lag8mo", "vap_mean_lag9mo", "vap_mean_lag10mo", "vap_mean_lag11mo", "vap_mean_lag12mo"]

vpd_mean_laggers = ["vpd_mean_lag1mo", "vpd_mean_lag2mo", "vpd_mean_lag3mo", "vpd_mean_lag4mo", "vpd_mean_lag5mo", "vpd_mean_lag6mo", 
                    "vpd_mean_lag7mo", "vpd_mean_lag8mo", "vpd_mean_lag9mo", "vpd_mean_lag10mo", "vpd_mean_lag11mo", "vpd_mean_lag12mo"]

vs_mean_laggers = ["vs_mean_lag1mo", "vs_mean_lag2mo", "vs_mean_lag3mo", "vs_mean_lag4mo", "vs_mean_lag5mo", "vs_mean_lag6mo", 
                   "vs_mean_lag7mo", "vs_mean_lag8mo", "vs_mean_lag9mo", "vs_mean_lag10mo", "vs_mean_lag11mo", "vs_mean_lag12mo"]  

In [172]:
df['aet_mean_lag'] = df[aet_mean_laggers].mean(axis=1)
df['def_mean_lag'] = df[def_mean_laggers].mean(axis=1)
df['pdsi_mean_lag'] = df[pdsi_mean_laggers].mean(axis=1)
df['pet_mean_lag'] = df[pet_mean_laggers].mean(axis=1)
df['pr_mean_lag'] = df[pr_mean_laggers].mean(axis=1)
df['ro_mean_lag'] = df[ro_mean_laggers].mean(axis=1)
df['soil_mean_lag'] = df[soil_mean_laggers].mean(axis=1)
df['srad_mean_lag'] = df[srad_mean_laggers].mean(axis=1)
df['tmmn_mean_lag'] = df[tmmn_mean_laggers].mean(axis=1)
df['tmmx_mean_lag'] = df[tmmx_mean_laggers].mean(axis=1)
df['vap_mean_lag'] = df[vap_mean_laggers].mean(axis=1)
df['vpd_mean_lag'] = df[vpd_mean_laggers].mean(axis=1)
df['vs_mean_lag'] = df[vs_mean_laggers].mean(axis=1)

In [174]:
climate_vars = ["aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", "soil_mean_lag", "srad_mean_lag",
                "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", "aet_mean", "def_mean", "pdsi_mean", "pet_mean", 
                "pr_mean", "ro_mean", "soil_mean", "srad_mean", "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean"] 

In [175]:
X_clim = df[climate_vars] #input features
y = df[target_var] #target var - shprev

In [176]:
X_climtrain, X_climtest, y_train, y_test = train_test_split(X_clim, y, test_size=0.2, random_state=42)

In [177]:
#Linear Regressor for climate, evaluation with RMSE direct.
clim_pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
clim_pipeline_LR.fit(X_climtrain, y_train)
y_clim_pred = clim_pipeline_LR.predict(X_climtest)
rmse = np.sqrt(mean_squared_error(y_test, y_clim_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 9.922752177375777


In [178]:
#Linear Regressor for climate, evaluation with Cross-val
rmse_scorer = make_scorer(mean_squared_error, squared=False)
cv_scores = cross_val_score(clim_pipeline_LR, X_clim, y, cv=10, scoring=rmse_scorer)
mean_rmse = np.mean(cv_scores)
print(f"Mean RMSE: {mean_rmse}")
warnings.filterwarnings("ignore", category=DeprecationWarning)

Mean RMSE: 13.071226247056359


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [179]:
#XGBoost Regressor for climate, evaluation with RMSE direct.
clim_pipeline_XGB = Pipeline([
    ('scaler', StandardScaler()),  
    ('regressor', XGBRegressor()) 
])
clim_pipeline_XGB.fit(X_climtrain, y_train)
y_clim_pred = clim_pipeline_XGB.predict(X_climtest)
rmse = np.sqrt(mean_squared_error(y_test, y_clim_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 10.346215246549914


In [180]:
#XGBoost Regressor for Climate, evaluation with Cross-val
rmse_scorer = make_scorer(mean_squared_error, squared=False)
cv_scores = cross_val_score(clim_pipeline_XGB, X_clim, y, cv=10, scoring=rmse_scorer)
mean_rmse = np.mean(cv_scores)
print(f"Mean RMSE: {mean_rmse}")
warnings.filterwarnings("ignore", category=DeprecationWarning)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 13.302056479078834


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [181]:
#Feature importance in Regressor
xgb_model = clim_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_climtrain.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df

,Feature,Importance
5,ro_mean_lag,0.169213
12,vs_mean_lag,0.108582
24,vpd_mean,0.069930
23,vap_mean,0.057453
25,vs_mean,0.040915
22,tmmx_mean,0.038939
20,srad_mean,0.038151
19,soil_mean,0.037287
11,vpd_mean_lag,0.037256
9,tmmx_mean_lag,0.033993


Vegetation Group

In [182]:
evi_mean_laggers = ["evi_mean_lag1mo", "evi_mean_lag2mo", "evi_mean_lag3mo", "evi_mean_lag4mo", "evi_mean_lag5mo", "evi_mean_lag6mo",
                    "evi_mean_lag7mo", "evi_mean_lag8mo", "evi_mean_lag9mo", "evi_mean_lag10mo", "evi_mean_lag11mo", "evi_mean_lag12mo"]

msavi_mean_laggers = ["msavi_mean_lag1mo", "msavi_mean_lag2mo", "msavi_mean_lag3mo", "msavi_mean_lag4mo", "msavi_mean_lag5mo", "msavi_mean_lag6mo", 
                      "msavi_mean_lag7mo", "msavi_mean_lag8mo", "msavi_mean_lag9mo", "msavi_mean_lag10mo", "msavi_mean_lag11mo", "msavi_mean_lag12mo"]

ndii_mean_laggers = ["ndii_mean_lag1mo", "ndii_mean_lag2mo", "ndii_mean_lag3mo", "ndii_mean_lag4mo", "ndii_mean_lag5mo", "ndii_mean_lag6mo", 
                     "ndii_mean_lag7mo", "ndii_mean_lag8mo", "ndii_mean_lag9mo", "ndii_mean_lag10mo", "ndii_mean_lag11mo", "ndii_mean_lag12mo"]

ndvi_mean_laggers = ["ndvi_mean_lag1mo", "ndvi_mean_lag2mo", "ndvi_mean_lag3mo", "ndvi_mean_lag4mo", "ndvi_mean_lag5mo", "ndvi_mean_lag6mo", 
                     "ndvi_mean_lag7mo", "ndvi_mean_lag8mo", "ndvi_mean_lag9mo", "ndvi_mean_lag10mo", "ndvi_mean_lag11mo", "ndvi_mean_lag12mo"]

ndwi_mean_laggers = ["ndwi_mean_lag1mo", "ndwi_mean_lag2mo", "ndwi_mean_lag3mo", "ndwi_mean_lag4mo", "ndwi_mean_lag5mo", "ndwi_mean_lag6mo",
                     "ndwi_mean_lag7mo", "ndwi_mean_lag8mo", "ndwi_mean_lag9mo", "ndwi_mean_lag10mo", "ndwi_mean_lag11mo", "ndwi_mean_lag12mo"]

savi_mean_laggers = ["savi_mean_lag1mo", "savi_mean_lag2mo", "savi_mean_lag3mo", "savi_mean_lag4mo", "savi_mean_lag5mo", "savi_mean_lag6mo",
                     "savi_mean_lag7mo", "savi_mean_lag8mo", "savi_mean_lag9mo", "savi_mean_lag10mo", "savi_mean_lag11mo", "savi_mean_lag12mo"]

In [183]:
df['evi_mean_lag'] = df[evi_mean_laggers].mean(axis=1)
df['msavi_mean_lag'] = df[msavi_mean_laggers].mean(axis=1)
df['ndii_mean_lag'] = df[ndii_mean_laggers].mean(axis=1)
df['ndvi_mean_lag'] = df[ndvi_mean_laggers].mean(axis=1)
df['ndwi_mean_lag'] = df[ndwi_mean_laggers].mean(axis=1)
df['savi_mean_lag'] = df[savi_mean_laggers].mean(axis=1)

In [184]:
veg_vars = ["evi_mean_lag", "msavi_mean_lag", "ndii_mean_lag", "ndvi_mean_lag", "ndwi_mean_lag", "savi_mean_lag", 
            "EVI_mean", "MSAVI_mean", "NDII_mean", "NDVI_mean", "NDWI_mean", "SAVI_mean"]
#what to do with the NaNs? 

In [185]:
for var in veg_vars: 
    print(f"{var} NaN count: {df[var].isna().sum()} ")

evi_mean_lag NaN count: 0 
msavi_mean_lag NaN count: 0 
ndii_mean_lag NaN count: 0 
ndvi_mean_lag NaN count: 0 
ndwi_mean_lag NaN count: 0 
savi_mean_lag NaN count: 0 
EVI_mean NaN count: 83 
MSAVI_mean NaN count: 83 
NDII_mean NaN count: 83 
NDVI_mean NaN count: 83 
NDWI_mean NaN count: 83 
SAVI_mean NaN count: 83 


In [186]:
veg_data_combined = df[veg_vars + [target_var]].dropna()
X_veg = veg_data_combined[veg_vars]
y = veg_data_combined[target_var]

In [187]:
X_vegtrain, X_vegtest, y_train, y_test = train_test_split(X_veg, y, test_size=0.2, random_state=42)

In [188]:
#Linear Regressor for veg, evaluation with RMSE direct - not introducing a scaler because they are all on the same scale 
veg_pipeline_LR = Pipeline([
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
veg_pipeline_LR.fit(X_vegtrain, y_train)
y_veg_pred = veg_pipeline_LR.predict(X_vegtest)
rmse = np.sqrt(mean_squared_error(y_test, y_veg_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 9.711678882194352


In [189]:
#Linear Regressor for veg, evaluation with Cross-val
def crosseval(pipeline_name, inputs, y_labels): 
    rmse_scorer = make_scorer(mean_squared_error, squared=False)
    cv_scores = cross_val_score(pipeline_name, inputs, y_labels, cv=10, scoring=rmse_scorer)
    mean_rmse = np.mean(cv_scores)
    print(f"Mean RMSE: {mean_rmse}")
    warnings.filterwarnings("ignore", category=DeprecationWarning)

crosseval(veg_pipeline_LR, X_veg, y)

Mean RMSE: 10.756538645823422


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [190]:
#XGBoost for veg, direct rmse
veg_pipeline_XGB = Pipeline([  
    ('regressor', XGBRegressor()) 
])
veg_pipeline_XGB.fit(X_vegtrain, y_train)
y_veg_pred = veg_pipeline_XGB.predict(X_vegtest)
rmse = np.sqrt(mean_squared_error(y_test, y_veg_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 9.892873576064636


In [191]:
#XGBoost veg, cross eval 
crosseval(veg_pipeline_XGB, X_veg, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 12.805280988019637


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [192]:
#feature importance
xgb_model = veg_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_vegtrain.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df

,Feature,Importance
3,ndvi_mean_lag,0.122041
8,NDII_mean,0.118685
10,NDWI_mean,0.116314
7,MSAVI_mean,0.102767
4,ndwi_mean_lag,0.095678
9,NDVI_mean,0.088652
11,SAVI_mean,0.076781
2,ndii_mean_lag,0.074055
6,EVI_mean,0.070194
1,msavi_mean_lag,0.056636


Before continuing with vegetation, I need to figure out what to do to deal with the NaNs. 

Geographic Variables

In [193]:
df["agricultural_area"] = df["area_rainfed_cropland"] + df["area_herbaceous_cover_cropland"] + df["area_irrigated_cropland"] 
df["forest_change"] = df["area_forest_lag1yr"] - df["area_forest"]

In [194]:
geo_vars = ["agricultural_area", "forest_change", "area_built_up"]

In [195]:
for var in geo_vars: 
    print(f"{var} NaN count: {df[var].isna().sum()} ")

agricultural_area NaN count: 496 
forest_change NaN count: 0 
area_built_up NaN count: 0 


In [196]:
agri_area = ["area_rainfed_cropland", "area_herbaceous_cover_cropland", "area_irrigated_cropland"]
for var in agri_area: 
    print(f"{var} NaN count: {df[var].isna().sum()} ")


area_rainfed_cropland NaN count: 0 
area_herbaceous_cover_cropland NaN count: 82 
area_irrigated_cropland NaN count: 455 


In [197]:
geo_data_combined = df[geo_vars + [target_var]].fillna(0)
X_geo = geo_data_combined[geo_vars]
y = geo_data_combined[target_var]

In [198]:
X_geotrain, X_geotest, y_train, y_test = train_test_split(X_geo, y, test_size=0.2, random_state=42)

In [199]:
df["agricultural_area"].isna().sum() 

496

In [200]:
df["area_built_up"].isna().sum()

0

In [201]:
df["forest_change"].isna().sum() 

0

In [202]:
#Linear Regressor for geo, evaluation with RMSE direct.
geo_pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
geo_pipeline_LR.fit(X_geotrain, y_train)
y_geo_pred = geo_pipeline_LR.predict(X_geotest)
rmse = np.sqrt(mean_squared_error(y_test, y_geo_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 10.141781093052046


In [203]:
#Linear regressor for geo, eval with crosseval
crosseval(geo_pipeline_LR, X_geo, y)

Mean RMSE: 11.518045477233793


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [204]:
#XGBoost for geo, direct rmse
geo_pipeline_XGB = Pipeline([  
    ('regressor', XGBRegressor()) 
])
geo_pipeline_XGB.fit(X_geotrain, y_train)
y_geo_pred = geo_pipeline_XGB.predict(X_geotest)
rmse = np.sqrt(mean_squared_error(y_test, y_geo_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 12.916686246765094


In [205]:
#XGBoost for geo, crosseval
crosseval(geo_pipeline_XGB, X_geo, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 14.381647146832345


In [206]:
#feature importance
xgb_model = geo_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_geotrain.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df

,Feature,Importance
2,area_built_up,0.370960
0,agricultural_area,0.351083
1,forest_change,0.277957


Human 

In [207]:
human_vars = ["schol_d", "total_pop", "total_pop_lag1yr", "total_pop_lag3yr", "gdp_mean", "gdp_mean_lag3yr", "gdp_mean_lag5yr"]

In [208]:
for var in human_vars: 
    print(f"{var} NaNs: {df[var].isna().sum()}")

schol_d NaNs: 0
total_pop NaNs: 0
total_pop_lag1yr NaNs: 0
total_pop_lag3yr NaNs: 0
gdp_mean NaNs: 0
gdp_mean_lag3yr NaNs: 0
gdp_mean_lag5yr NaNs: 0


In [209]:
df["gdp_mean_lag"] = df[["gdp_mean_lag3yr", "gdp_mean_lag5yr"]].mean(axis=1)

In [210]:
pop_change_1yr = ((df["total_pop_lag1yr"] - df["total_pop"])/df["total_pop_lag1yr"]) * 100 
pop_change_1yr.mean()

-2.5654473805792577

In [211]:
df["pop_change_3yr"] = ((df["total_pop_lag3yr"] - df["total_pop"])/df["total_pop_lag3yr"]) * 100 
df["pop_change_3yr"].mean()

-7.901295371501186

In [212]:
def encode_population_change(change):
    if change >= 5:
        return 'decrease'
    elif change <= -5:
        return 'increase'
    else:
        return 'stays the same'


In [213]:
df['pop_change_category'] = df['pop_change_3yr'].apply(encode_population_change)

In [214]:
df["pop_change_category"].value_counts()

pop_change_category
increase          1124
stays the same     386
Name: count, dtype: int64

In [215]:
human_vars = ["total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag"]

In [216]:
X_hum = df[human_vars]
y = df[target_var]

In [217]:
X_hum

,total_pop,pop_change_category,gdp_mean,gdp_mean_lag
0,107906.177237,stays the same,1.600000e+07,1.642201e+07
1,153361.402931,increase,2.940000e+07,2.987605e+07
2,166798.479845,increase,2.560000e+07,2.603080e+07
3,140934.339622,increase,2.530000e+07,2.579169e+07
4,140934.339622,increase,2.530000e+07,2.579169e+07
...,...,...,...,...
1505,101847.408054,increase,5.384014e+06,4.344233e+06
1506,273941.131112,increase,2.290000e+07,1.835703e+07
1507,156430.687760,increase,1.730000e+07,1.395032e+07
1508,147410.779260,stays the same,1.310000e+07,1.054927e+07


In [218]:
X_humtrain, X_humtest, y_train, y_test = train_test_split(X_hum, y, test_size=0.2, random_state=42)

In [219]:
hum_numerical_features = [col for col in X_humtrain.columns if col != 'pop_change_category']
hum_categorical_features = ['pop_change_category']
hum_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), hum_numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), hum_categorical_features)  # OneHotEncode categorical feature
    ]
)

In [220]:
#Linear Regressor for hum, evaluation with RMSE direct.
hum_pipeline_LR = Pipeline([
    ('preprocessor', hum_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', LinearRegression())  # Step 2: Linear Regression
])
hum_pipeline_LR.fit(X_humtrain, y_train)
y_hum_pred = hum_pipeline_LR.predict(X_humtest)
rmse = np.sqrt(mean_squared_error(y_test, y_hum_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 10.292539610953911


In [221]:
crosseval(hum_pipeline_LR, X_hum, y)

Mean RMSE: 11.79989377709385


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [222]:
#XGBoost for hum, direct rmse
hum_pipeline_XGB = Pipeline([
    ('preprocessor', hum_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', XGBRegressor())  # Step 2: Linear Regression
])
hum_pipeline_XGB.fit(X_humtrain, y_train)
y_hum_pred = hum_pipeline_XGB.predict(X_humtest)
rmse = np.sqrt(mean_squared_error(y_test, y_hum_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 12.57187202783863


In [223]:
crosseval(hum_pipeline_XGB, X_hum, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 13.047362239312866


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [224]:
preprocessor = hum_pipeline_XGB.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()
xgb_model = hum_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df


,Feature,Importance
2,num__gdp_mean_lag,0.328012
1,num__gdp_mean,0.280008
0,num__total_pop,0.223049
3,cat__pop_change_category_increase,0.168931
4,cat__pop_change_category_stays the same,0.000000


Overall S.Haematobium Model

In [225]:
sh_vars = [
    'aspect', 'bedrock_depth_mean', 'bulk_density_20cm_mean', 'cec_20cm_mean', 'clay_20cm_mean', 
    'damDist', 'elevation', 'hnd', 'nitrogen_20cm_mean', 'organic_carbon_20cm_mean', 'ph_20cm_mean',
    'sand_20cm_mean', 'silt_20cm_mean', 'slope', 'soilWater', 'stone_20cm_mean', 'total_carbon_20cm_mean',
    'upa', 'waterDist', "aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", 
    "soil_mean_lag", "srad_mean_lag", "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", 
    "aet_mean", "def_mean", "pdsi_mean", "pet_mean", "pr_mean", "ro_mean", "soil_mean", "srad_mean", 
    "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean", "evi_mean_lag", "msavi_mean_lag", "ndii_mean_lag", "ndvi_mean_lag", 
    "ndwi_mean_lag", "savi_mean_lag", "EVI_mean", "MSAVI_mean", "NDII_mean", "NDVI_mean", "NDWI_mean", "SAVI_mean", "agricultural_area",
    "forest_change", "area_built_up", "total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag"]




In [226]:
sh_total = [
    'aspect', 'bedrock_depth_mean', 'bulk_density_20cm_mean', 'cec_20cm_mean', 'clay_20cm_mean', 
    'damDist', 'elevation', 'hnd', 'nitrogen_20cm_mean', 'organic_carbon_20cm_mean', 'ph_20cm_mean',
    'sand_20cm_mean', 'silt_20cm_mean', 'slope', 'soilWater', 'stone_20cm_mean', 'total_carbon_20cm_mean',
    'upa', 'waterDist', "aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", 
    "soil_mean_lag", "srad_mean_lag", "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", 
    "aet_mean", "def_mean", "pdsi_mean", "pet_mean", "pr_mean", "ro_mean", "soil_mean", "srad_mean", 
    "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean", "evi_mean_lag", "msavi_mean_lag", "ndii_mean_lag", "ndvi_mean_lag", 
    "ndwi_mean_lag", "savi_mean_lag", "EVI_mean", "MSAVI_mean", "NDII_mean", "NDVI_mean", "NDWI_mean", "SAVI_mean", "agricultural_area",
    "forest_change", "area_built_up", "total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag", "sh_prev"
]

In [227]:
df_cleaned = df[sh_total].dropna() 

In [228]:
X_sh = df_cleaned[sh_vars]
y = df_cleaned[target_var]

In [229]:
X_sh

,aspect,bedrock_depth_mean,bulk_density_20cm_mean,cec_20cm_mean,clay_20cm_mean,damDist,elevation,hnd,nitrogen_20cm_mean,organic_carbon_20cm_mean,...,NDVI_mean,NDWI_mean,SAVI_mean,agricultural_area,forest_change,area_built_up,total_pop,pop_change_category,gdp_mean,gdp_mean_lag
32,183.325919,199.998470,1.412489,10.229578,21.115993,46844.425547,248.292468,10.481229,1.651412,11.666908,...,0.265673,-0.282836,0.198551,16.015684,1.094556,21.979507,201863.291253,increase,14700000.0,1.527270e+07
33,182.784741,199.998470,1.411513,10.225347,21.126795,46501.673160,249.224401,10.660945,1.650517,11.674590,...,0.265752,-0.282873,0.198584,15.997085,1.083994,22.338219,201393.123546,increase,14500000.0,1.509216e+07
34,183.217584,199.998470,1.409945,10.223845,21.166215,46648.751319,250.154609,10.742454,1.651843,11.701085,...,0.260511,-0.278126,0.195958,15.527148,0.873300,22.499839,200975.959582,increase,14300000.0,1.488384e+07
35,181.579420,199.998470,1.410897,10.221216,21.124946,46125.318953,249.706008,10.786566,1.649135,11.677880,...,0.267269,-0.284202,0.199359,15.885762,1.145445,23.368959,201134.781290,increase,14500000.0,1.503780e+07
36,181.579420,199.998470,1.410897,10.221216,21.124946,46125.318953,249.706008,10.786566,1.649135,11.677880,...,0.267269,-0.284202,0.199359,15.885762,1.145445,23.368959,201134.781290,increase,14500000.0,1.503780e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,171.397774,199.942883,1.272063,11.235303,24.985543,58267.294801,408.529399,41.570153,1.722089,13.799234,...,0.786956,-0.704606,0.545470,14.182259,2.214575,33.995516,331276.819790,increase,37400000.0,2.994470e+07
1506,172.548618,199.978153,1.276121,10.958453,24.456480,55342.672736,400.657967,43.761110,1.729774,13.170437,...,0.784700,-0.700568,0.549154,27.547012,6.084565,30.315688,273941.131112,increase,22900000.0,1.835703e+07
1507,179.351368,199.999975,1.407290,9.879511,20.807100,43775.095235,228.699718,12.693525,1.646505,11.509531,...,0.783950,-0.698550,0.534724,101.989168,3.055017,17.073475,156430.687760,increase,17300000.0,1.395032e+07
1508,165.346038,199.990415,1.289833,10.524264,23.651749,53708.898851,363.918758,38.565175,1.733319,12.470827,...,0.785734,-0.698745,0.556012,35.480976,9.840979,16.616431,147410.779260,stays the same,13100000.0,1.054927e+07


In [230]:
sh_numerical_features = [col for col in X_shtrain.columns if col != 'pop_change_category']
sh_categorical_features = ['pop_change_category']

sh_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), sh_numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), sh_categorical_features)  # OneHotEncode categorical feature
    ]
)

In [231]:
#Linear Regressor for hum, evaluation with RMSE direct.
sh_pipeline_LR = Pipeline([
    ('preprocessor', sh_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', LinearRegression())  # Step 2: Linear Regression
])

In [232]:
crosseval(sh_pipeline_LR, X_sh, y)

Mean RMSE: 12.869232115915219


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [235]:
sh_pipeline_XGB = Pipeline([
    ('preprocessor', sh_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', XGBRegressor())  # Step 2: Linear Regression
])
crosseval(sh_pipeline_XGB, X_sh, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 11.278530389784034


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Mansoni Analysis

In [241]:
df = pd.read_csv('sm.csv')

Geophysical Group

In [257]:
df

,aspect,bedrock_depth_mean,bio12,bio13,bio14,bio15,bio16,bio17,bio18,bio19,...,area_grassland_lag1yr,area_sparse_veg_lag1yr,area_wetlands_lag1yr,area_built_up_lag1yr,area_bare_lag1yr,area_open_water_lag1yr,gdp_mean,total_pop_lag3yr,gdp_mean_lag3yr,gdp_mean_lag5yr
0,183.604181,200.000000,1508.830955,280.060468,21.733899,57.115762,646.825496,120.533794,337.455792,317.999699,...,0.129867,0.012451,0.370666,10.853481,0.000889,0.096916,1.600000e+07,105437.180646,1.636735e+07,1.647667e+07
1,185.425880,200.000000,1377.345726,241.037560,16.313812,57.442422,585.446491,93.790261,325.758583,328.468667,...,0.116440,0.008889,0.597385,24.045927,0.000889,0.314707,2.940000e+07,140894.190245,2.985771e+07,2.989439e+07
2,190.578224,200.000000,1446.334404,261.036755,19.465539,56.627543,615.037975,109.811994,328.340045,321.447973,...,0.092564,0.010671,0.376095,22.010989,0.000889,0.276497,2.560000e+07,156083.530521,2.600184e+07,2.605977e+07
3,186.927523,200.000000,1426.579422,254.018940,18.599338,56.953311,606.025688,103.754191,327.599493,330.564783,...,0.087157,0.011558,0.573416,21.549089,0.003556,0.314721,2.530000e+07,130071.696107,2.575529e+07,2.582808e+07
4,186.927523,200.000000,1426.579422,254.018940,18.599338,56.953311,606.025688,103.754191,327.599493,330.564783,...,0.087157,0.011558,0.573416,21.549089,0.003556,0.314721,2.530000e+07,130071.696107,2.575529e+07,2.582808e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,175.894180,199.994981,1394.855386,265.084019,12.526002,63.791606,626.972542,73.165778,259.101799,622.791588,...,0.013294,0.185405,4.564108,13.912753,0.021273,7.009784,5.384014e+06,96657.256388,4.686037e+06,4.002429e+06
1639,172.548618,199.978153,1525.434371,285.713904,13.252617,64.114136,689.668872,76.351853,285.847580,681.678590,...,0.006207,0.014183,0.201890,29.890035,NaN,0.001774,2.290000e+07,259012.837724,1.991745e+07,1.679662e+07
1640,179.351368,199.999975,1395.601557,252.328301,14.692797,57.583724,569.940314,89.073386,301.566096,520.616641,...,0.003551,0.563824,38.775236,17.261871,0.073709,55.075660,1.730000e+07,145999.885540,1.508205e+07,1.281859e+07
1641,165.346038,199.990415,1478.862700,278.517084,12.898370,63.998466,665.964303,74.576893,277.598148,664.910081,...,0.004432,0.008864,0.154243,15.891669,NaN,0.001774,1.310000e+07,143062.583056,1.141841e+07,9.680133e+06


In [258]:
df_geophys_sm = df[geophysical_vars + [target_var]].dropna()

In [260]:
X_geophys = df_geophys_sm[geophysical_vars] #input features
y = df_geophys_sm[target_var] #target var - shprev

In [261]:
geophys_pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
crosseval(geophys_pipeline_LR, X_geophys, y)

Mean RMSE: 12.404193620375672


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [262]:
geophys_pipeline_XGB = Pipeline([
    ('scaler', StandardScaler()),  
    ('regressor', XGBRegressor()) 
])
crosseval(geophys_pipeline_XGB, X_geophys, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 12.85530025631619


In [274]:
geophys_pipeline_XGB.fit(X_geophys, y)
xgb_model = geophys_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_geophys.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
3,cec_20cm_mean,0.124447
12,silt_20cm_mean,0.091554
6,elevation,0.089715
13,slope,0.068779
8,nitrogen_20cm_mean,0.065361
2,bulk_density_20cm_mean,0.060573
16,total_carbon_20cm_mean,0.056439
18,waterDist,0.055556
14,soilWater,0.047268
4,clay_20cm_mean,0.045543


Climate Group

In [275]:
df['aet_mean_lag'] = df[aet_mean_laggers].mean(axis=1)
df['def_mean_lag'] = df[def_mean_laggers].mean(axis=1)
df['pdsi_mean_lag'] = df[pdsi_mean_laggers].mean(axis=1)
df['pet_mean_lag'] = df[pet_mean_laggers].mean(axis=1)
df['pr_mean_lag'] = df[pr_mean_laggers].mean(axis=1)
df['ro_mean_lag'] = df[ro_mean_laggers].mean(axis=1)
df['soil_mean_lag'] = df[soil_mean_laggers].mean(axis=1)
df['srad_mean_lag'] = df[srad_mean_laggers].mean(axis=1)
df['tmmn_mean_lag'] = df[tmmn_mean_laggers].mean(axis=1)
df['tmmx_mean_lag'] = df[tmmx_mean_laggers].mean(axis=1)
df['vap_mean_lag'] = df[vap_mean_laggers].mean(axis=1)
df['vpd_mean_lag'] = df[vpd_mean_laggers].mean(axis=1)
df['vs_mean_lag'] = df[vs_mean_laggers].mean(axis=1)

In [276]:
climate_vars = ["aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", "soil_mean_lag", "srad_mean_lag",
                "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", "aet_mean", "def_mean", "pdsi_mean", "pet_mean", 
                "pr_mean", "ro_mean", "soil_mean", "srad_mean", "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean"] 

In [277]:
df_clim_sm = df[climate_vars + [target_var]].dropna()

In [278]:
X_clim = df_clim_sm[climate_vars] #input features
y = df_clim_sm[target_var] #target var - shprev

In [279]:
clim_pipeline_LR = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
crosseval(clim_pipeline_LR, X_clim, y)

Mean RMSE: 14.846244885791368


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [280]:
clim_pipeline_XGB = Pipeline([
    ('scaler', StandardScaler()),    # Step 1: Scaling
    ('regressor', XGBRegressor()) # Step 2: Linear Regression
])
crosseval(clim_pipeline_XGB, X_clim, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 12.399680148451235


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [281]:
clim_pipeline_XGB.fit(X_clim, y)
xgb_model = clim_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_clim.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
5,ro_mean_lag,0.140114
12,vs_mean_lag,0.109713
10,vap_mean_lag,0.067302
2,pdsi_mean_lag,0.055889
20,srad_mean,0.053656
16,pet_mean,0.044786
23,vap_mean,0.043691
11,vpd_mean_lag,0.042350
17,pr_mean,0.041539
7,srad_mean_lag,0.040104


Vegetation Group

In [282]:
df['evi_mean_lag'] = df[evi_mean_laggers].mean(axis=1)
df['msavi_mean_lag'] = df[msavi_mean_laggers].mean(axis=1)
df['ndii_mean_lag'] = df[ndii_mean_laggers].mean(axis=1)
df['ndvi_mean_lag'] = df[ndvi_mean_laggers].mean(axis=1)
df['ndwi_mean_lag'] = df[ndwi_mean_laggers].mean(axis=1)
df['savi_mean_lag'] = df[savi_mean_laggers].mean(axis=1)

In [283]:
df_veg_sm = df[veg_vars + [target_var]].dropna()

In [285]:
X_veg = df_veg_sm[veg_vars] #input features
y = df_veg_sm[target_var] #target var - shprev

In [286]:
veg_pipeline_LR = Pipeline([
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
crosseval(veg_pipeline_LR, X_veg, y)

Mean RMSE: 11.05504323712319


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [287]:
veg_pipeline_XGB = Pipeline([
    ('regressor', XGBRegressor()) # Step 2: Linear Regression
])
crosseval(veg_pipeline_XGB, X_veg, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 12.114107506966404


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [288]:
veg_pipeline_XGB.fit(X_veg, y)
xgb_model = veg_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_veg.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
8,NDII_mean,0.130338
2,ndii_mean_lag,0.100952
4,ndwi_mean_lag,0.093211
9,NDVI_mean,0.088496
3,ndvi_mean_lag,0.087279
1,msavi_mean_lag,0.085169
11,SAVI_mean,0.083079
7,MSAVI_mean,0.079936
10,NDWI_mean,0.073769
6,EVI_mean,0.065448


Geography Group

In [289]:
df["agricultural_area"] = df["area_rainfed_cropland"] + df["area_herbaceous_cover_cropland"] + df["area_irrigated_cropland"] 
df["forest_change"] = df["area_forest_lag1yr"] - df["area_forest"]

In [292]:
df_geo_sm = df[geo_vars + [target_var]].dropna()

In [293]:
X_geo = df_geo_sm[geo_vars] #input features
y = df_geo_sm[target_var] #target var - shprev

In [294]:
geo_pipeline_LR = Pipeline([
    ('regressor', LinearRegression()) # Step 2: Linear Regression
])
crosseval(geo_pipeline_LR, X_geo, y)

Mean RMSE: 12.39820530415273


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [295]:
geo_pipeline_XGB = Pipeline([
    ('regressor', XGBRegressor()) # Step 2: Linear Regression
])
crosseval(geo_pipeline_XGB, X_geo, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 13.743487043114673


In [296]:
geo_pipeline_XGB.fit(X_geo, y)
xgb_model = geo_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_geo.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
2,area_built_up,0.409881
1,forest_change,0.350353
0,agricultural_area,0.239766


Human Group

In [298]:
df["gdp_mean_lag"] = df[["gdp_mean_lag3yr", "gdp_mean_lag5yr"]].mean(axis=1)
pop_change_1yr = ((df["total_pop_lag1yr"] - df["total_pop"])/df["total_pop_lag1yr"]) * 100 
df["pop_change_3yr"] = ((df["total_pop_lag3yr"] - df["total_pop"])/df["total_pop_lag3yr"]) * 100 
df['pop_change_category'] = df['pop_change_3yr'].apply(encode_population_change)

human_vars = ["total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag"]


In [299]:
df_hum_sm = df[human_vars + [target_var]].dropna()

In [300]:
X_hum = df_hum_sm[human_vars]
y = df_hum_sm[target_var]

In [301]:
hum_numerical_features = [col for col in X_hum.columns if col != 'pop_change_category']
hum_categorical_features = ['pop_change_category']
hum_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), hum_numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), hum_categorical_features)  # OneHotEncode categorical feature
    ]
)

In [302]:
hum_pipeline_LR = Pipeline([
    ('preprocessor', hum_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', LinearRegression())  # Step 2: Linear Regression
])
crosseval(hum_pipeline_LR, X_hum, y)

Mean RMSE: 12.15323744476059


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [303]:
hum_pipeline_XGB = Pipeline([
    ('preprocessor', hum_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', XGBRegressor())  # Step 2: Linear Regression
])
crosseval(hum_pipeline_XGB, X_hum, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 13.595095780747078


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [305]:
preprocessor = hum_pipeline_XGB.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()
xgb_model = hum_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df


,Feature,Importance
2,num__gdp_mean_lag,0.314728
1,num__gdp_mean,0.273382
3,cat__pop_change_category_increase,0.226212
0,num__total_pop,0.185677
4,cat__pop_change_category_stays the same,0.000000


Overall Mansoni

In [306]:
sm_vars = [
    'aspect', 'bedrock_depth_mean', 'bulk_density_20cm_mean', 'cec_20cm_mean', 'clay_20cm_mean', 
    'damDist', 'elevation', 'hnd', 'nitrogen_20cm_mean', 'organic_carbon_20cm_mean', 'ph_20cm_mean',
    'sand_20cm_mean', 'silt_20cm_mean', 'slope', 'soilWater', 'stone_20cm_mean', 'total_carbon_20cm_mean',
    'upa', 'waterDist', "aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", 
    "soil_mean_lag", "srad_mean_lag", "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", 
    "aet_mean", "def_mean", "pdsi_mean", "pet_mean", "pr_mean", "ro_mean", "soil_mean", "srad_mean", 
    "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean", "evi_mean_lag", "msavi_mean_lag", "ndii_mean_lag", "ndvi_mean_lag", 
    "ndwi_mean_lag", "savi_mean_lag", "EVI_mean", "MSAVI_mean", "NDII_mean", "NDVI_mean", "NDWI_mean", "SAVI_mean", "agricultural_area",
    "forest_change", "area_built_up", "total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag"]

In [307]:
sm_total = [
    'aspect', 'bedrock_depth_mean', 'bulk_density_20cm_mean', 'cec_20cm_mean', 'clay_20cm_mean', 
    'damDist', 'elevation', 'hnd', 'nitrogen_20cm_mean', 'organic_carbon_20cm_mean', 'ph_20cm_mean',
    'sand_20cm_mean', 'silt_20cm_mean', 'slope', 'soilWater', 'stone_20cm_mean', 'total_carbon_20cm_mean',
    'upa', 'waterDist', "aet_mean_lag", "def_mean_lag", "pdsi_mean_lag", "pet_mean_lag", "pr_mean_lag", "ro_mean_lag", 
    "soil_mean_lag", "srad_mean_lag", "tmmn_mean_lag", "tmmx_mean_lag", "vap_mean_lag", "vpd_mean_lag", "vs_mean_lag", 
    "aet_mean", "def_mean", "pdsi_mean", "pet_mean", "pr_mean", "ro_mean", "soil_mean", "srad_mean", 
    "tmmn_mean", "tmmx_mean", "vap_mean", "vpd_mean", "vs_mean", "evi_mean_lag", "msavi_mean_lag", "ndii_mean_lag", "ndvi_mean_lag", 
    "ndwi_mean_lag", "savi_mean_lag", "EVI_mean", "MSAVI_mean", "NDII_mean", "NDVI_mean", "NDWI_mean", "SAVI_mean", "agricultural_area",
    "forest_change", "area_built_up", "total_pop", "pop_change_category", "gdp_mean", "gdp_mean_lag", "sh_prev"
]

In [308]:
df_cleaned_sm = df[sm_total].dropna()

In [309]:
X_sm = df_cleaned_sm[sm_vars]
y = df_cleaned_sm[target_var]

In [310]:
sm_numerical_features = [col for col in X_sm.columns if col != 'pop_change_category']
sm_categorical_features = ['pop_change_category']

sm_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), sm_numerical_features),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), sm_categorical_features)  # OneHotEncode categorical feature
    ]
)

In [311]:
sm_pipeline_LR = Pipeline([
    ('preprocessor', sm_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', LinearRegression())  # Step 2: Linear Regression
])
crosseval(sm_pipeline_LR, X_sm, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean RMSE: 13.266816265167048


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

In [312]:
sm_pipeline_XGB = Pipeline([
    ('preprocessor', sm_preprocessor),    # Step 1: Preprocessing (Scaling and OneHotEncoding)
    ('regressor', XGBRegressor())  # Step 2: Linear Regression
])
crosseval(sm_pipeline_XGB, X_sm, y)

c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated i

Mean RMSE: 10.966497343381125


c:\Users\syeda\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [314]:
preprocessor = sm_pipeline_XGB.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()
xgb_model = sm_pipeline_XGB.named_steps['regressor']
importances = xgb_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
importance_df

,Feature,Importance
21,num__pdsi_mean_lag,0.114330
22,num__pet_mean_lag,0.057953
33,num__def_mean,0.055715
4,num__clay_20cm_mean,0.040894
40,num__tmmn_mean,0.040833
...,...,...
43,num__vpd_mean,0.001655
36,num__pr_mean,0.001374
16,num__total_carbon_20cm_mean,0.000747
63,cat__pop_change_category_increase,0.000000
